In [1]:
import pandas as pd
import numpy as np

# === preprocessing ==
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# === deep learning ==
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model




print("All libraries loaded successfully!")
print("TensorFlow version:", tf.__version__)

All libraries loaded successfully!
TensorFlow version: 2.20.0


In [4]:
import sys
print(sys.executable)

c:\Users\lenovo\.virtualenvs\DL_Projects--W8vBDqt\Scripts\python.exe


In [2]:
df_medical = pd.read_csv('Srms_ims.csv')

In [3]:
df_medical.head()

,Patient ID,Age,Gender,Region,Occupation,Date of Diagnosis,Symptoms,Sputum Smear Test,GeneXpert Test,Chest X-ray Results,...,Malnutrition,Diabetes,HIV,Chronic Lung Disease,Smoking Status,Alcohol Consumption,Living Conditions,Access to Healthcare,City,Region Code
0,1,12,Female,Rural,Other,310,Weight loss,Negative,Negative,Normal,...,Yes,Yes,No,No,Non-smoker,Non-drinker,Average,No,Bogura,34
1,2,30,Other,Urban,Other,1538,Fever,Negative,Positive,Abnormal,...,No,No,No,No,Non-smoker,Drinker,Rich,Yes,Bogura,39
2,3,13,Other,Rural,Student,2217,Night Sweats,Positive,Positive,Abnormal,...,No,No,No,No,Ex-smoker,Non-drinker,Rich,No,Sylhet,41
3,4,47,Male,Urban,Worker,1887,Fever,Positive,Negative,Abnormal,...,Yes,Yes,No,Yes,Non-smoker,Drinker,Poor,No,Narayanganj,39
4,5,35,Male,Rural,Other,933,Cough,Positive,Negative,Abnormal,...,No,Yes,No,Yes,Ex-smoker,Drinker,Average,Yes,Jessore,48


In [4]:
# check the null values
df_medical.isna().sum()

Patient ID                  0
Age                         0
Gender                      0
Region                      0
Occupation                  0
Date of Diagnosis           0
Symptoms                    0
Sputum Smear Test           0
GeneXpert Test              0
Chest X-ray Results         0
Treatment Start Date        0
Treatment Type              0
Duration of Treatment       0
Drug Resistance             0
Treatment Outcome           0
Relapse                     0
Mortality                   0
Complications            4078
Malnutrition                0
Diabetes                    0
HIV                         0
Chronic Lung Disease        0
Smoking Status              0
Alcohol Consumption         0
Living Conditions           0
Access to Healthcare        0
City                        0
Region Code                 0
dtype: int64

In [5]:
len(df_medical)

20000

In [6]:
df_medical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Patient ID             20000 non-null  int64 
 1   Age                    20000 non-null  int64 
 2   Gender                 20000 non-null  object
 3   Region                 20000 non-null  object
 4   Occupation             20000 non-null  object
 5   Date of Diagnosis      20000 non-null  int64 
 6   Symptoms               20000 non-null  object
 7   Sputum Smear Test      20000 non-null  object
 8   GeneXpert Test         20000 non-null  object
 9   Chest X-ray Results    20000 non-null  object
 10  Treatment Start Date   20000 non-null  int64 
 11  Treatment Type         20000 non-null  object
 12  Duration of Treatment  20000 non-null  int64 
 13  Drug Resistance        20000 non-null  object
 14  Treatment Outcome      20000 non-null  object
 15  Relapse            

In [7]:
df = df_medical.copy()

In [9]:
df.drop(["Patient ID", "City", "complications"], axis=1, inplace=True, errors='ignore')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Age                    20000 non-null  int64 
 1   Gender                 20000 non-null  object
 2   Region                 20000 non-null  object
 3   Occupation             20000 non-null  object
 4   Date of Diagnosis      20000 non-null  int64 
 5   Symptoms               20000 non-null  object
 6   Sputum Smear Test      20000 non-null  object
 7   GeneXpert Test         20000 non-null  object
 8   Chest X-ray Results    20000 non-null  object
 9   Treatment Start Date   20000 non-null  int64 
 10  Treatment Type         20000 non-null  object
 11  Duration of Treatment  20000 non-null  int64 
 12  Drug Resistance        20000 non-null  object
 13  Treatment Outcome      20000 non-null  object
 14  Relapse                20000 non-null  object
 15  Mortality          

In [14]:
df.shape

(20000, 26)

In [11]:
# select categorical columns
cat_cols = df.select_dtypes(include=['object']).columns

print("Categorical Columns:\n", cat_cols)

Categorical Columns:
 Index(['Gender', 'Region', 'Occupation', 'Symptoms', 'Sputum Smear Test',
       'GeneXpert Test', 'Chest X-ray Results', 'Treatment Type',
       'Drug Resistance', 'Treatment Outcome', 'Relapse', 'Mortality',
       'Complications', 'Malnutrition', 'Diabetes', 'HIV',
       'Chronic Lung Disease', 'Smoking Status', 'Alcohol Consumption',
       'Living Conditions', 'Access to Healthcare'],
      dtype='object')


In [15]:
num_cols = df.select_dtypes(exclude=['object']).columns
print("Numerical Columns", num_cols)

Numerical Columns Index(['Age', 'Date of Diagnosis', 'Treatment Start Date',
       'Duration of Treatment', 'Region Code'],
      dtype='object')


In [12]:
for col in cat_cols:
    print("\n==============================")
    print(f"Column: {col}")
    print(df[col].value_counts())


Column: Gender
Gender
Other     6733
Female    6663
Male      6604
Name: count, dtype: int64

Column: Region
Region
Rural    10022
Urban     9978
Name: count, dtype: int64

Column: Occupation
Occupation
Laborer    4066
Student    4045
Other      4017
Farmer     3953
Worker     3919
Name: count, dtype: int64

Column: Symptoms
Symptoms
Cough           5135
Fever           5039
Night Sweats    4923
Weight loss     4903
Name: count, dtype: int64

Column: Sputum Smear Test
Sputum Smear Test
Positive    10154
Negative     9846
Name: count, dtype: int64

Column: GeneXpert Test
GeneXpert Test
Positive    10030
Negative     9970
Name: count, dtype: int64

Column: Chest X-ray Results
Chest X-ray Results
Abnormal    10014
Normal       9986
Name: count, dtype: int64

Column: Treatment Type
Treatment Type
Drug-resistant TB treatment    10067
DOTS                            9933
Name: count, dtype: int64

Column: Drug Resistance
Drug Resistance
Yes    10114
No      9886
Name: count, dtype: int64

C

In [13]:
treatment_mapping = {
    'Drug-resistant TB treatment': 'DR-TB',
    'DOTS': 'DOTS'
}

df['Treatment Type'] = df['Treatment Type'].replace(treatment_mapping)

In [ ]:
target_col1 = 'Chest X-ray Results'
X1_inputs = 

In [25]:
df['Symptoms'].value_counts()

Symptoms
fever           2569
cough           2525
weight loss     2447
night sweats    2401
Name: count, dtype: int64

In [ ]:
df["Symptoms"]

In [12]:
# convert to lowercase for easy matching
df["Symptoms"] = df["Symptoms"].str.lower()

# Chest X-ray related symptoms
df["dry_cough"] = df["Symptoms"].str.contains("dry cough").astype(int)
df["chest_pain"] = df["Symptoms"].str.contains("chest pain").astype(int)
df["mild_fever"] = df["Symptoms"].str.contains("mild fever").astype(int)

# GeneXpert related symptoms
df["sputum_cough"] = df["Symptoms"].str.contains("sputum").astype(int)
df["evening_fever"] = df["Symptoms"].str.contains("evening fever").astype(int)
df["weight_loss"] = df["Symptoms"].str.contains("weight loss").astype(int)
df["loss_appetite"] = df["Symptoms"].str.contains("loss of appetite").astype(int)
df["night_sweats"] = df["Symptoms"].str.contains("night sweats").astype(int)


In [13]:
# target values :  GeneXpert Test [], Chest X-ray Results
# Input Parameters :  all relevant fields
# Functional Model
# feature enginering 
# drop irrelevant columns
df.drop(["Patient ID", "City"], axis=1, inplace=True)

# fill missing values
df["Complications"] = df["Complications"].fillna("None")


In [14]:
df["Date of Diagnosis"] = pd.to_datetime(df["Date of Diagnosis"])
df["Treatment Start Date"] = pd.to_datetime(df["Treatment Start Date"])


In [15]:
df["disease_duration_days"] = (
    df["Treatment Start Date"] - df["Date of Diagnosis"]
).dt.days

In [16]:
# remove invalid rows
df = df[df["disease_duration_days"] >= 0]

In [17]:
# convert to months
df["disease_duration_months"] = df["disease_duration_days"] / 30

In [18]:
features_M1 = [
    "Age", "Gender",
    "Sputum Smear Test",
    "sputum_cough",
    "evening_fever",
    "weight_loss",
    "loss_appetite",
    "night_sweats",
    "Diabetes",
    "HIV",
    "Alcohol Consumption",
    "Smoking Status",
    "Malnutrition",
    "disease_duration_months"
]

X_M1 = df[features_M1]
y_M1 = df["GeneXpert Test"]

In [26]:
y_M1

0        Negative
2        Positive
3        Negative
4        Negative
6        Negative
           ...   
19987    Positive
19990    Positive
19993    Positive
19996    Positive
19998    Negative
Name: GeneXpert Test, Length: 9942, dtype: object

In [19]:
from sklearn.preprocessing import LabelEncoder

le_target1 = LabelEncoder()
y_M1_le = le_target1.fit_transform(df["GeneXpert Test"])

le_target2 = LabelEncoder()
y_M2_le = le_target2.fit_transform(df["Chest X-ray Results"])


In [20]:
features_M2 = [
    "Age", "Gender",
    "dry_cough",
    "chest_pain",
    "mild_fever",
    "Smoking Status",
    "Chronic Lung Disease",
    "Living Conditions",
    "Region Code",
    "Malnutrition"
]

X_M2 = df[features_M2]
y_M2 = df["Chest X-ray Results"]


In [21]:
X_M1[0:5]

,Age,Gender,Sputum Smear Test,sputum_cough,evening_fever,weight_loss,loss_appetite,night_sweats,Diabetes,HIV,Alcohol Consumption,Smoking Status,Malnutrition,disease_duration_months
0,12,Female,Negative,0,0,1,0,0,Yes,No,Non-drinker,Non-smoker,Yes,0.0
2,13,Other,Positive,0,0,0,0,1,No,No,Non-drinker,Ex-smoker,No,0.0
3,47,Male,Positive,0,0,0,0,0,Yes,No,Drinker,Non-smoker,Yes,0.0
4,35,Male,Positive,0,0,0,0,0,Yes,No,Drinker,Ex-smoker,No,0.0
6,65,Female,Negative,0,0,0,0,0,Yes,Yes,Non-drinker,Non-smoker,No,0.0


In [ ]:
X_M1.head()

In [ ]:
X_M1.info()

In [22]:
from sklearn.preprocessing import LabelEncoder

encoders_M1 = {}

categorical_cols_M1 = X_M1.select_dtypes(include="object").columns

for col in categorical_cols_M1:
    le = LabelEncoder()
    X_M1[col] = le.fit_transform(X_M1[col])
    encoders_M1[col] = le


C:\Users\lenovo\AppData\Local\Temp\ipykernel_2852\4130345784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_M1[col] = le.fit_transform(X_M1[col])
C:\Users\lenovo\AppData\Local\Temp\ipykernel_2852\4130345784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_M1[col] = le.fit_transform(X_M1[col])
C:\Users\lenovo\AppData\Local\Temp\ipykernel_2852\4130345784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [23]:
X_M2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9942 entries, 0 to 19998
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Age                   9942 non-null   int64 
 1   Gender                9942 non-null   object
 2   dry_cough             9942 non-null   int64 
 3   chest_pain            9942 non-null   int64 
 4   mild_fever            9942 non-null   int64 
 5   Smoking Status        9942 non-null   object
 6   Chronic Lung Disease  9942 non-null   object
 7   Living Conditions     9942 non-null   object
 8   Region Code           9942 non-null   int64 
 9   Malnutrition          9942 non-null   object
dtypes: int64(5), object(5)
memory usage: 854.4+ KB


In [ ]:
encoders_M2 = {}

categorical_cols_M2 = X_M2.select_dtypes(include="object").columns

for col in categorical_cols_M2:
    le = LabelEncoder()
    X_M2[col] = le.fit_transform(X_M2[col])
    encoders_M2[col] = le

In [ ]:
import joblib

joblib.dump(encoders_M1, "encoders_M1.pkl")
joblib.dump(encoders_M2, "encoders_M2.pkl")

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler1 = StandardScaler()
X1_scaled = scaler1.fit_transform(X_M1)

In [ ]:
scaler2 = StandardScaler()
X2_scaled = scaler1.fit_transform(X_M2)

In [ ]:
joblib.dump(scaler1, "scaler_M1.pkl")
joblib.dump(scaler2, "scaler_M2.pkl")

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1_scaled, y_M1_le, test_size=0.2, random_state=42
)

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2_scaled,y_M2_le, test_size=0.2, random_state=42
)


In [ ]:
X1_train[0:2]

In [24]:
import joblib

features_M1 = X_M1.columns.tolist()
features_M2 = X_M2.columns.tolist()

joblib.dump(features_M1, "features_M1.pkl")
joblib.dump(features_M2, "features_M2.pkl")

['features_M2.pkl']

In [ ]:

def build_model(input_dim):

    inputs = Input(shape=(input_dim,))

    x = Dense(128, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(32, activation='relu')(x)

    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model



In [ ]:
# build model M1
model_M1 = build_model(X1_train.shape[1])

In [ ]:
model_M1.summary()

In [ ]:
# train
history_M1 = model_M1.fit(
    X1_train, y1_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32,
    verbose=1
)

In [ ]:
# build model M1
model_M2 = build_model(X2_train.shape[1])

In [ ]:
# train
history_M2 = model_M2.fit(
    X2_train, y2_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32,
    verbose=1
)

In [ ]:
pred1 = (model_M1.predict(X1_test) > 0.5).astype(int)
pred2 = (model_M2.predict(X2_test) > 0.5).astype(int)

In [ ]:
pred1 = (model_M1.predict(X1_test) > 0.5).astype(int)
pred2 = (model_M2.predict(X2_test) > 0.5).astype(int)

In [ ]:
import numpy as np

final_pred = np.logical_and(pred1, pred2).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

print("GeneXpert Model Accuracy:", accuracy_score(y1_test, pred1))
print("Chest X-ray Model Accuracy:", accuracy_score(y2_test, pred2))

print("\nFinal TB Prediction Accuracy:")
print(accuracy_score(y1_test, final_pred))

print("\nClassification Report:")
print(classification_report(y1_test, final_pred))

In [ ]:
model_M1.save("model_M1.h5")
model_M2.save("model_M2.h5")

In [28]:
print(y_M1_le)
print(y_M2_le)

[0 1 0 ... 1 1 0]
[1 0 0 ... 0 1 1]
